# Aula 14 - Cálculo de PCM usando PySCF

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, dft
from pyscf.solvent import pcm
from pyscf.geomopt import geometric_solver
from pyscf.tools import cubegen # para exportar aquivos cube

Baseado em https://github.com/pyscf/pyscf/blob/master/examples/solvent/05-pcm.py

# Glicina e tautômeros

## Otimizando a geometria no vácuo

Definindo a geometria inicial com conjunto de base

In [2]:
aminoacid = gto.M(atom='''
        O   -2.61517   0.14434  -0.52409
        C   -1.47791   0.45532  -0.23939
        O   -0.85915   1.46529  -0.79124
        C   -0.64608  -0.28047   0.79178
        N    0.77595  -0.38543   0.40918
        H    0.04515   1.46444  -0.42620
        H   -1.07166  -1.28155   0.90583
        H   -0.73742   0.26337   1.73609
        H    0.85780  -0.88722  -0.47349
        H    1.28537  -0.92052   1.10915''',
              basis='6-31+g*', verbose=1)

zwitterion = gto.M(atom='''
        N   0.80069   0.95989  -0.01502
        C   2.28229   1.04570  -0.01205
        C   2.75010   1.30567  -1.41047
        O   1.89488   1.39987  -2.26935
        O   3.94347   1.40102  -1.57562
        H   0.41459   0.78110   0.92045
        H   0.53515   0.21886  -0.68231
        H   0.44056   1.84213  -0.41111
        H   2.75669   0.11558   0.31025
        H   2.65463   1.86920   0.60213''',
              basis='6-31+g*', verbose=1)

In [3]:
mol = aminoacid
mol_string = 'glycine_aminoacid'  # 'glycine_aminoacid' ou 'glycine_zwitterion'

In [4]:
mf = dft.RKS(mol,xc='b3lyp')
new_mol = geometric_solver.optimize(mf)

geometric-optimize called with the following command line:
/home/elvis/Programs/miniforge3/envs/quantum/lib/python3.13/site-packages/ipykernel_launcher.py --f=/run/user/1000/jupyter/runtime/kernel-v3a2aae6c00f4b6d6763039cd46c8c0387066fbcf1.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%

Revelando as novas coordenadas

In [5]:
new_mol.atom_coords(unit='Angstrom')

array([[-2.73969688,  0.5301345 , -0.14886717],
       [-1.52994055,  0.54167059, -0.14856515],
       [-0.80071965,  1.40861041, -0.86946866],
       [-0.69048189, -0.42785176,  0.70196171],
       [ 0.73860373, -0.3254908 ,  0.37000187],
       [ 0.14484134,  1.18126749, -0.68286273],
       [-1.10479354, -1.43362883,  0.57007794],
       [-0.85388189, -0.15135552,  1.74906338],
       [ 1.05189011, -1.08867312, -0.22393843],
       [ 1.33209447, -0.30809603,  1.19378547]])

Colocando as novas coordenadas na molécula geometria original

In [6]:
mol.set_geom_(new_mol.atom_coords(unit='Angstrom'))

Fazendo cálculo de densidade eletrônica e energia

In [7]:
mf.kernel()

-284.4367461432464

Calculando a densidade eletrônica

In [8]:
dm = mf.make_rdm1() # densidade eletrônica na presença do solvente

Exportando a densidade eletrônica

In [9]:
cubegen.density(mol, f'rho_{mol_string}.cube', dm)

array([[[7.33864002e-08, 8.19508306e-08, 9.13826858e-08, ...,
         1.08056751e-07, 9.70144609e-08, 8.68718837e-08],
        [8.83865698e-08, 9.88098941e-08, 1.10303822e-07, ...,
         1.20472463e-07, 1.08073583e-07, 9.67035783e-08],
        [1.06298263e-07, 1.18978641e-07, 1.32981149e-07, ...,
         1.33980131e-07, 1.20081904e-07, 1.07359789e-07],
        ...,
        [2.24551582e-06, 2.54737287e-06, 2.88129849e-06, ...,
         1.43681657e-07, 1.20780343e-07, 1.01348683e-07],
        [1.96976564e-06, 2.23084446e-06, 2.51905698e-06, ...,
         1.27855834e-07, 1.07617709e-07, 9.04162484e-08],
        [1.72267152e-06, 1.94778252e-06, 2.19578563e-06, ...,
         1.13513538e-07, 9.56683137e-08, 8.04748067e-08]],

       [[8.50111188e-08, 9.49958589e-08, 1.06003079e-07, ...,
         1.30635433e-07, 1.17320175e-07, 1.05086167e-07],
        [1.02383071e-07, 1.14531021e-07, 1.27939684e-07, ...,
         1.45569055e-07, 1.30627684e-07, 1.16921840e-07],
        [1.23137370e-07, 

Exportando o potencial eletrostático 

In [10]:
cubegen.mep(mol, f'esp_{mol_string}.cube', dm)

array([[[-0.00754247, -0.00762756, -0.00771144, ..., -0.00124687,
         -0.00112531, -0.00100775],
        [-0.00791477, -0.00800925, -0.00810264, ..., -0.00140161,
         -0.00127296, -0.00114862],
        [-0.00830145, -0.008406  , -0.0085096 , ..., -0.00156227,
         -0.00142617, -0.0012947 ],
        ...,
        [-0.02728483, -0.02782516, -0.02836661, ..., -0.01034095,
         -0.00994907, -0.00957041],
        [-0.02677167, -0.02728627, -0.02780107, ..., -0.01023251,
         -0.00985065, -0.00948138],
        [-0.02625994, -0.02674983, -0.02723908, ..., -0.01012262,
         -0.00975063, -0.00939062]],

       [[-0.00736827, -0.00745015, -0.00753081, ..., -0.00098068,
         -0.00086265, -0.00074866],
        [-0.00774336, -0.00783477, -0.00792509, ..., -0.00113216,
         -0.00100702, -0.00088625],
        [-0.00813335, -0.00823501, -0.00833572, ..., -0.00128967,
         -0.00115707, -0.00102915],
        ...,
        [-0.02779505, -0.02835677, -0.02892006, ..., -

## Otimizando a geometria usando modelo PCM para solvente

Definindo solvente e modelo PCM

In [11]:
cm = pcm.PCM(mol)
cm.eps = 78.3553  # constante dielétrica da água
cm.method = 'SS(V)PE' # or IEF-PCM, C-PCM, SS(V)PE, COSMO, see https://manual.q-chem.com/5.4/topic_pcm-em.html
cm.lebedev_order = 29 # lebedev grids on the cavity surface, lebedev_order=29  <--> # of grids = 302

Otimizando a geometria agora na presença do solvente

In [12]:
mf_sol = dft.RKS(mol, xc='b3lyp').PCM(cm)
mol_solvent = geometric_solver.optimize(mf_sol)

geometric-optimize called with the following command line:
/home/elvis/Programs/miniforge3/envs/quantum/lib/python3.13/site-packages/ipykernel_launcher.py --f=/run/user/1000/jupyter/runtime/kernel-v3a2aae6c00f4b6d6763039cd46c8c0387066fbcf1.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%

In [13]:
mol_solvent.atom_coords(unit='Angstrom')

array([[-2.74432756,  0.58009314, -0.09012004],
       [-1.52407338,  0.53501859, -0.12188159],
       [-0.79923476,  1.38059202, -0.85790745],
       [-0.68418507, -0.4721188 ,  0.66986972],
       [ 0.73932868, -0.28985846,  0.36080214],
       [ 0.15070247,  1.09880332, -0.68722614],
       [-1.0456961 , -1.47614842,  0.4237245 ],
       [-0.88953041, -0.30322829,  1.7323466 ],
       [ 1.12804373, -1.09385282, -0.12569667],
       [ 1.29490376, -0.13341117,  1.19746588]])

Fazendo cálculo de densidade eletrônica e energia

In [14]:
mf_sol.kernel()

-284.5164996592472

Cálculando a matriz densidade

In [15]:
dm = mf_sol.make_rdm1() # densidade eletrônica na presença do solvente

Exportando a densidade eletrônica em solvente

In [16]:
cubegen.density(mol_solvent, f'rho_{mol_string}_inwater.cube', dm)

array([[[1.29251664e-07, 1.48211274e-07, 1.69288684e-07, ...,
         1.13556070e-06, 1.06081249e-06, 9.87675408e-07],
        [1.46606819e-07, 1.68262925e-07, 1.92367526e-07, ...,
         1.25218374e-06, 1.16902758e-06, 1.08775792e-06],
        [1.65630707e-07, 1.90302297e-07, 2.17808244e-07, ...,
         1.37828096e-06, 1.28592730e-06, 1.19577043e-06],
        ...,
        [7.01660896e-05, 7.27531141e-05, 7.51015125e-05, ...,
         2.29379500e-06, 2.08137907e-06, 1.88022941e-06],
        [6.75240101e-05, 7.01528029e-05, 7.25629907e-05, ...,
         1.98811041e-06, 1.80612039e-06, 1.63342377e-06],
        [6.46685090e-05, 6.73176454e-05, 6.97689138e-05, ...,
         1.71266010e-06, 1.55784064e-06, 1.41060184e-06]],

       [[1.45936371e-07, 1.66767120e-07, 1.89825244e-07, ...,
         1.29841328e-06, 1.21618897e-06, 1.13530843e-06],
        [1.65861226e-07, 1.89687663e-07, 2.16084357e-07, ...,
         1.42808535e-06, 1.33683382e-06, 1.24718349e-06],
        [1.87892150e-07, 

Exportando o potencial eletrostático em solvente

In [17]:
cubegen.mep(mol_solvent, f'esp_{mol_string}_inwater.cube', dm)

array([[[0.06954815, 0.06999022, 0.0704302 , ..., 0.07689785,
         0.07652633, 0.07614727],
        [0.06967096, 0.07011444, 0.0705558 , ..., 0.07726047,
         0.07688662, 0.07650495],
        [0.06977355, 0.07021782, 0.07065992, ..., 0.07761211,
         0.07723629, 0.07685235],
        ...,
        [0.04004673, 0.03983699, 0.0396255 , ..., 0.06242714,
         0.06245755, 0.06246856],
        [0.04011099, 0.03992039, 0.0397291 , ..., 0.06200859,
         0.06203643, 0.06204559],
        [0.04017986, 0.04000758, 0.03983551, ..., 0.0615918 ,
         0.0616171 , 0.06162444]],

       [[0.07066675, 0.07113173, 0.07159478, ..., 0.07812886,
         0.07773573, 0.07733508],
        [0.07080687, 0.07127385, 0.07173888, ..., 0.07851553,
         0.07811945, 0.07771559],
        [0.07092641, 0.07139475, 0.07186109, ..., 0.07889129,
         0.07849262, 0.07808587],
        ...,
        [0.04023383, 0.04001567, 0.03979544, ..., 0.06336794,
         0.06338935, 0.06339087],
        [0.0